# Q&A Application using GraphDB

In [1]:
# !pip install python-dotenv 

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

## Creating the graph

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

## Loading the dataset

In [5]:
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [6]:
graph.query(movies_query)

[]

## Graph Schema

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


## Chain

In [11]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [12]:
response = {"query":"List all movies directed by Martin Scorsese.","result":"The movies directed by Martin Scorsese are Casino and Taxi Driver.","intermediate_steps":[{"query":"MATCH (d:Person {name: 'Martin Scorsese'})-[:DIRECTED]->(m:Movie) RETURN m.title"},{"context":[{"m.title":"Casino"},{"m.title":"Taxi Driver"}]}]}

In [14]:
response["result"]

'The movies directed by Martin Scorsese are Casino and Taxi Driver.'

In [15]:
response["intermediate_steps"][0]["query"]

"MATCH (d:Person {name: 'Martin Scorsese'})-[:DIRECTED]->(m:Movie) RETURN m.title"